In [2]:
import sys
sys.path.insert(0, '../../') 
from project.data.datasets.ARG_KP_2021.processor import load_dataset
import pandas as pd

### load all arguments with quality score and ground truth kps 

In [37]:
#train_df = pd.read_csv(r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\ARG_KP_2021\train_complete.csv", index_col=None)
#dev_df =  pd.read_csv(r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\ARG_KP_2021\dev_complete.csv", index_col=None)
test_df =  pd.read_csv(r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\ARG_KP_2021\test_complete.csv", index_col=None)

In [38]:
print(len(test_df))
test_df.head(2)

3923


,idx,arg_id,argument,topic,stance,MACE-P,key_point_id,key_point,label,label_strict,label_relaxed,text,sentence1,sentence2
0,0,arg_0_0,Routine child vaccinations isn t mandatory since children don t spread the virus,Routine child vaccinations should be mandatory,-1,0.714195,kp_0_0,Routine child vaccinations or their side effects are dangerous,0.0,0.0,0.0,Routine child vaccinations isn t mandatory since children don t spread the virus Routine child vaccinations or their side effects are dangerous Routine child vaccinations should be mandatory,[CLS] Routine child vaccinations isn t mandatory since children don t spread the virus [SEP] Routine child vaccinations should be mandatory [SEP],[CLS] Routine child vaccinations or their side effects are dangerous [SEP] Routine child vaccinations should be mandatory [SEP]
1,1,arg_0_0,Routine child vaccinations isn t mandatory since children don t spread the virus,Routine child vaccinations should be mandatory,-1,0.714195,kp_0_1,Mandatory vaccination contradicts basic rights,0.0,0.0,0.0,Routine child vaccinations isn t mandatory since children don t spread the virus Mandatory vaccination contradicts basic rights Routine child vaccinations should be mandatory,[CLS] Routine child vaccinations isn t mandatory since children don t spread the virus [SEP] Routine child vaccinations should be mandatory [SEP],[CLS] Mandatory vaccination contradicts basic rights [SEP] Routine child vaccinations should be mandatory [SEP]


In [5]:
# import the potential kps, was prepare in a separate noteook (prepare_potential_kps.ipnyb)
all_pot_sent_df = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\raw\ArgKP_2021_shared_task\page_rank\pagerank-generated-test-keypoints.csv', index_col=None)

In [39]:
# to be able to compare, we take only arguments with positive single label
test_ds_labeled = test_df[test_df['label'] == 1]
test_ds_labeled = test_ds_labeled.sample(frac=1).drop_duplicates(subset='arg_id')

### create cross args dataset

Get every possible pair to get matching score from best model. 

In [10]:
# all_pot_sent_df has potential kps sentences and test_ds_labeled has the arguments we want to generate kps for them

def get_paired_args(arguments_df, potential_kps, topic, stance):
    args_ids = []
    args = []
    kps = []
    q_scores = []
    
    filtered_arguments_df = arguments_df[arguments_df['topic'] == topic]
    filtered_potential_kps_df = potential_kps[potential_kps['topic'] == topic]
    
    filtered_arguments_df = filtered_arguments_df[filtered_arguments_df['stance'] == stance]
    filtered_potential_kps_df = filtered_potential_kps_df[filtered_potential_kps_df['stance'] == stance]
    
    unique_args_id = filtered_arguments_df['arg_id'].unique()
    
    for arg_id in unique_args_id:
        arg_row = filtered_arguments_df[filtered_arguments_df["arg_id"] == arg_id].iloc[0]
        for  index, s_row in filtered_potential_kps_df.iterrows():
            if arg_row["argument"] == s_row["key_point"]:
                continue
            args_ids.append(arg_id)
            args.append(arg_row["argument"])
            kps.append(s_row["key_point"])
    
    res_df = pd.DataFrame({
        "arg_id": args_ids,
        "argument": args,
        "key_point": kps
    })
    
    res_df['topic'] = topic
    res_df['stance'] = stance
    return res_df

topics = test_ds_labeled['topic'].unique()
stances = [-1, 1]

cross_pairs_df = None
for topic in topics:
    for stance in stances:
        if cross_pairs_df is None:
            cross_pairs_df = get_paired_args(test_ds_labeled, all_pot_sent_df,  topic, stance)
        else:
            cross_pairs_df = pd.concat([cross_pairs_df, get_paired_args(test_ds_labeled, all_pot_sent_df, topic, stance)])

In [11]:
cross_pairs_df.head(2)

,arg_id,argument,key_point,topic,stance
0,arg_2_28,The United States could have better management...,The poorest in society don't have access to e...,The USA is a good country to live in,-1
1,arg_2_28,The United States could have better management...,The United States has long been a hotbed of r...,The USA is a good country to live in,-1


In [12]:
len(cross_pairs_df)

6611

In [13]:
cross_pairs_df.to_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_cross_pairs_df_pr.csv', index=False)

### load best model 

Here we load best matcher model and score the pairs

In [14]:
import pandas as pd
test_cross_pairs_df = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_cross_pairs_df_pr.csv', index_col=None)

In [15]:
len(test_cross_pairs_df)

6611

In [16]:
test_cross_pairs_df['idx'] = test_cross_pairs_df.index
test_cross_pairs_df.head(2)

,arg_id,argument,key_point,topic,stance,idx
0,arg_2_28,The United States could have better management...,The poorest in society don't have access to e...,The USA is a good country to live in,-1,0
1,arg_2_28,The United States could have better management...,The United States has long been a hotbed of r...,The USA is a good country to live in,-1,1


In [17]:
# process data to work with the classifier
from project.data.datasets.ARG_KP_2021.processor import preprocessing

test_cross_pairs_df['argument'] = test_cross_pairs_df['argument'].apply(lambda x: preprocessing(x))
test_cross_pairs_df['key_point'] = test_cross_pairs_df['key_point'].apply(lambda x: preprocessing(x))
test_cross_pairs_df['topic'] = test_cross_pairs_df['topic'].apply(lambda x: preprocessing(x))

test_cross_pairs_df['sentence1'] = '[CLS] ' + test_cross_pairs_df['argument'] + ' [SEP] ' + test_cross_pairs_df['topic'] + ' [SEP]'
test_cross_pairs_df['sentence2'] = '[CLS] ' + test_cross_pairs_df['key_point'] + ' [SEP] ' + test_cross_pairs_df['topic'] + ' [SEP]'

In [18]:
test_cross_pairs_df.to_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_cross_pairs_clean_pr_df.csv', index=False)

In [19]:
# load best model
import torch
torch.cuda.empty_cache()
from project.models.baselines.classification_model import ClassificationModel
from project.models.common.nlp_configs import HFTransformerDataConfig
from transformers import AutoTokenizer
from project.data.nlp_data_module import NLPDataModule

test_data_path = r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_cross_pairs_clean_df.csv'
model_checkpoint_path = r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\models\good-models\epoch=1-step=807.ckpt"
model = ClassificationModel.load_from_checkpoint(checkpoint_path=model_checkpoint_path)
data_cfg = HFTransformerDataConfig(**{
    "batch_size": 32,
    "num_workers": 4,
    "train_file": test_data_path,
    "test_file": test_data_path
})
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model.model.config._name_or_path, use_fast=True)
datamodule = NLPDataModule(cfg=data_cfg, tokenizer=tokenizer)
datamodule.setup("test")
test_dataset = datamodule.ds["test"]

C:\Users\Bilal\anaconda3\envs\cryptocurrencies-kpa\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using custom data configuration

  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\Bilal\.cache\huggingface\datasets\csv\default-63492bacee025feb\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a\cache-c2c998aa52b9c7ad.arrow


  0%|          | 0/9 [00:00<?, ?ba/s]

In [20]:
test_dataset.column_names

['arg_id',
 'argument',
 'attention_mask',
 'attention_mask_sent1',
 'attention_mask_sent2',
 'idx',
 'input_ids',
 'input_ids_sent1',
 'input_ids_sent2',
 'key_point',
 'kp_q_score',
 'sentence1',
 'sentence2',
 'stance',
 'topic']

In [21]:
# score match score for every possible pair
BATCH_SIZE = 32

test_ds_df = test_dataset.to_pandas()
test_ds_df.set_index('idx')
test_ds_df['score'] = -1

model.to('cuda')

def add_preds(examples):
    similarity, _, output = model._forward(examples, predict_with_gpu=True)
    for i, idx in enumerate(examples["idx"]):
        if output is not None:
            score = output[i][1].item()
        else:
            score = similarity[i].item()
        test_ds_df.iat[idx.item(), test_ds_df.columns.get_loc('score')] = score

test_dataset.map(lambda examples: add_preds(examples), batched=True, batch_size=BATCH_SIZE)



  0%|          | 0/280 [00:00<?, ?ba/s]

Dataset({
    features: ['arg_id', 'argument', 'attention_mask', 'attention_mask_sent1', 'attention_mask_sent2', 'idx', 'input_ids', 'input_ids_sent1', 'input_ids_sent2', 'key_point', 'kp_q_score', 'sentence1', 'sentence2', 'stance', 'topic'],
    num_rows: 8960
})

In [22]:
test_ds_df.head(2)
test_ds_df.to_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_cross_pairs_with_match_score_pr.csv', index=False)

In [40]:
import pandas as pd
test_ds_df = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_cross_pairs_with_match_score_pr.csv', index_col=None)

In [41]:
print(len(test_ds_df))
test_ds_df.head(1)

8960


,arg_id,argument,attention_mask,attention_mask_sent1,attention_mask_sent2,idx,input_ids,input_ids_sent1,input_ids_sent2,key_point,kp_q_score,sentence1,sentence2,stance,topic,score
0,arg_2_58,recently there was a lot of political problems that I m afraid it will make it hard to live there plus some regions don t have a lot of attention from the government,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0\n 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],0,[ 0 10975 7454 104 742 682 89 21 10 319 9 559\n 1272 14 38 475 6023 24 40 146 24 543 7 697\n 89 2704 103 3806 218 326 33 10 319 9 1437 1503\n 31 5 168 646 3388 510 742 20 2805 16 10 205\n 247 7 697 11 646 3388 510 742 2 2 10975 7454\n 104 742 20 2805 16 10 182 27703 247 1437 24 473\n 45 694 10547 474 575 50 1265 13 63 2286 1437 646\n 3388 510 742 20 2805 16 10 205 247 7 697 11\n 646 3388 510 742 2 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1],[ 0 10975 7454 104 742 682 89 21 10 319 9 559\n 1272 14 38 475 6023 24 40 146 24 543 7 697\n 89 2704 103 3806 218 326 33 10 319 9 1437 1503\n 31 5 168 646 3388 510 742 20 2805 16 10 205\n 247 7 697 11 646 3388 510 742 2 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1],[ 0 10975 7454 104 742 20 2805 16 10 182 27703 247\n 1437 24 473 45 694 10547 474 575 50 1265 13 63\n 2286 1437 646 3388 510 742 20 2805 16 10 205 247\n 7 697 11 646 3388 510 742 2 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1],The USA is a very capitalist country it does not provide universal health care or education for its citizens,0.897261,[CLS] recently there was a lot of political problems that I m afraid it will make it hard to live there plus some regions don t have a lot of attention from the government [SEP] The USA is a good country to live in [SEP],[CLS] The USA is a very capitalist country it does not provide universal health care or education for its citizens [SEP] The USA is a good country to live in [SEP],-1,The USA is a good country to live in,0.18933


In [ ]:
test_ds_df[['sentence1', 'sentence2', 'stance', 'topic', 'score']][test_ds_df['score']> 0.5].sort_values(by='score', ascending=True).head(20)

In [43]:
# get top kps based on their number of matches (based on a threshold)
threshold = 0.5
result_df = test_ds_df[test_ds_df['score'] > threshold]
# choose best possible kp considering quality + match + rule-based
# first for every arg_id, get top 20 matches
# result_df = test_ds_df.groupby(by="arg_id").apply(lambda df: df.sample(frac=1, random_state=42).sort_values(by="score", ascending=False).head(1)).reset_index(drop=True)
# add combined average score quality+match/2 
# result_df['combined_score'] = (result_df['kp_q_score'] + result_df['score'] )/ 2
# second take the arg with top combined score from the 20 matches
# result_df = result_df.groupby(by="arg_id").apply(lambda df: df.sample(frac=1, random_state=42).sort_values(by="combined_score", ascending=False).head(1)).reset_index(drop=True)

In [44]:
len(result_df)

3087

In [45]:
print(len(result_df))
result_df.head(2)

3087


,arg_id,argument,attention_mask,attention_mask_sent1,attention_mask_sent2,idx,input_ids,input_ids_sent1,input_ids_sent2,key_point,kp_q_score,sentence1,sentence2,stance,topic,score
4,arg_2_58,recently there was a lot of political problems that I m afraid it will make it hard to live there plus some regions don t have a lot of attention from the government,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0\n 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],4,[ 0 10975 7454 104 742 682 89 21 10 319 9 559\n 1272 14 38 475 6023 24 40 146 24 543 7 697\n 89 2704 103 3806 218 326 33 10 319 9 1437 1503\n 31 5 168 646 3388 510 742 20 2805 16 10 205\n 247 7 697 11 646 3388 510 742 2 2 10975 7454\n 104 742 20 2805 16 45 10 205 247 7 697 11\n 142 89 16 98 203 168 797 1437 646 3388 510 742\n 20 2805 16 10 205 247 7 697 11 646 3388 510\n 742 2 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1],[ 0 10975 7454 104 742 682 89 21 10 319 9 559\n 1272 14 38 475 6023 24 40 146 24 543 7 697\n 89 2704 103 3806 218 326 33 10 319 9 1437 1503\n 31 5 168 646 3388 510 742 20 2805 16 10 205\n 247 7 697 11 646 3388 510 742 2 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1],[ 0 10975 7454 104 742 20 2805 16 45 10 205 247\n 7 697 11 142 89 16 98 203 168 797 1437 646\n 3388 510 742 20 2805 16 10 205 247 7 697 11\n 646 3388 510 742 2 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1],The USA is not a good country to live in because there is so much government control,0.781724,[CLS] recently there was a lot of political problems that I m afraid it will make it hard to live there plus some regions don t have a lot of attention from the government [SEP] The USA is a good country to live in [SEP],[CLS] The USA is not a good country to live in because there is so much government control [SEP] The USA is a good country to live in [SEP],-1,The USA is a good country to live in,0.584242
8,arg_2_58,recently there was a lot of political problems that I m afraid it will make it hard to live there plus some regions don t have a lot of attention from the government,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1\n 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1\n 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],8,[ 0 10975 7454 104 742 682 89 21 10 319 9 559\n 1272 14 38 475 6023 24 40 146 24 543 7 697\n 89 2704 103 3806 218 326 33 10 319 9 1437 1503\n 31 5 168 646 3388 510 742 20 2805 16 10 205\n 247 7 697 11 646 

In [54]:
# get the top 10 most frequent KPs 

def get_topic_kps_counts(df):
    unique_topics = df['topic'].unique()
    unique_stances = [-1, 1]
    
    topics = []
    stances = []
    kps = []
    counts = []
    q_scores = []
    for t in unique_topics:
        for stance in unique_stances:
            f_df = df[df['topic'] == t]
            f_df = f_df[f_df['stance'] == stance]   
            unique_kps = f_df['key_point'].unique()
           
            for kp in unique_kps:
                topics.append(t)
                stances.append(stance)
                kps.append(kp)
                #q_scores.append(f_df[f_df['key_point'] == kp].iloc[0]['kp_q_score'])
                counts.append(len(f_df[f_df['key_point'] == kp]))

    return pd.DataFrame({
            "topic": topics,
            "stance": stances,
            "key_point": kps,
            #"q_score": q_scores,
            "counts": counts
        })
    

In [47]:
counts_df = get_topic_kps_counts(result_df)

In [48]:
counts_df.head(2)

,topic,stance,key_point,q_score,counts
0,The USA is a good country to live in,-1,The USA is not a good country to live in because there is so much government control,0.781724,13
1,The USA is a good country to live in,-1,The USA is a greatly divided country that can make it difficult to succeed,0.725220,4


In [51]:
counst_df = counts_df.groupby(by=["topic", "stance"]).apply(lambda df: df.sample(frac=1, random_state=42).sort_values(by=["counts"], ascending=(False)).head(4)).reset_index(drop=True)

In [52]:
len(counst_df)

24

In [53]:
pd.set_option('display.max_colwidth', None)
counst_df.head(24)

,topic,stance,key_point,q_score,counts
0,Routine child vaccinations should be mandatory,-1,child vaccinations should not be mandatory to avoid any bad effects on them,0.773095,58
1,Routine child vaccinations should be mandatory,-1,Routine child vaccinations should not be mandatory because children may not bear the side effects of it,0.871068,54
2,Routine child vaccinations should be mandatory,-1,Children should not be vaccinated because they can have serious side effects,0.865730,54
3,Routine child vaccinations should be mandatory,-1,Vaccines in children should not be mandatory because they can have consequences for their health in the future,0.885895,45
4,Routine child vaccinations should be mandatory,1,Child vaccination should be mandatory to prevent fatal disease in children I think it is a very good decision,0.865199,104
5,Routine child vaccinations should be mandatory,1,Children must be vaccinated because it protects against most infectious diseases,0.898835,103
6,Routine child vaccinations should be mandatory,1,Children s vaccines should be frequent since they protect our children from any virus and disease,0.913100,102
7,Routine child vaccinations should be mandatory,1,vaccines must be compulsory for children because in this way we prevent later diseases in infants,0.922973,99
8,Social media platforms should be regulated by the government,-1,Social media should not be regulated by the government because it would infringe the right to free speech,0.936327,72
9,Social media platforms should be regulated by the government,-1,social networks are a means of free expression it should not be regulated,0.937492,67


In [55]:
# comparing topics with labeled test dataset
test_ds_labeled_counts = get_topic_kps_counts(test_ds_labeled)

In [56]:
test_ds_labeled.columns

Index(['idx', 'arg_id', 'argument', 'topic', 'stance', 'MACE-P',
       'key_point_id', 'key_point', 'label', 'label_strict', 'label_relaxed',
       'text', 'sentence1', 'sentence2'],
      dtype='object')

In [57]:
# for every topic and stance
test_ds_labeled_counts = test_ds_labeled_counts.groupby(by=["topic", "stance"]).apply(lambda df: df.sample(frac=1, random_state=42).sort_values(by="counts", ascending=False).head(5)).reset_index(drop=True)

In [58]:
test_ds_labeled_counts.head(24)

,topic,stance,key_point,counts
0,Routine child vaccinations should be mandatory,-1,Routine child vaccinations or their side effects are dangerous,51
1,Routine child vaccinations should be mandatory,-1,The parents and not the state should decide,32
2,Routine child vaccinations should be mandatory,-1,Mandatory vaccination contradicts basic rights,12
3,Routine child vaccinations should be mandatory,-1,Routine child vaccinations are not necessary to keep children healthy,11
4,Routine child vaccinations should be mandatory,1,Children should not suffer from preventable diseases,51
5,Routine child vaccinations should be mandatory,1,Routine child vaccinations should be mandatory to prevent virus disease spreading,40
6,Routine child vaccinations should be mandatory,1,Routine child vaccinations are effective,21
7,Routine child vaccinations should be mandatory,1,Routine child vaccinations are necessary to protect others,18
8,Routine child vaccinations should be mandatory,1,Child vaccination saves lives,16
9,Social media platforms should be regulated by the government,-1,Social media regulation harm freedom of speech and other democratic rights,49


# compare with the ground truth
 now we should compare result_df with test_ds_labeled , each having 500 arg_id

In [59]:
import rouge
# evaluate to ground truth using rouge
evaluator = rouge.Rouge(metrics=['rouge-l'],
                           limit_length=True,
                           length_limit_type='words',
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)

In [60]:
# comparing test_ds_labeled_counts with counst_df
# for every topic, stance => compare predicted kp with every labeled kp and take max score
import numpy as np
from itertools import product
unique_topics = test_ds_labeled['topic'].unique()
unique_stances = [-1, 1]

topics = []
stances = []
scores = []

for t in unique_topics:
    for s in unique_stances:
        test_df = test_ds_labeled_counts[test_ds_labeled_counts['topic'] == t]
        test_df = test_df[test_df['stance'] == s]
        res_df = counst_df[counst_df['topic'] == t]
        res_df = res_df[res_df['stance'] == s]
        
        unique_predicted_kps = res_df['key_point'].unique()
        unique_labeled_kps = test_df['key_point'].unique()
        kp_scores = []
        for p_kp, l_kp in product(unique_predicted_kps, unique_labeled_kps) :
                score = evaluator.get_scores(p_kp, l_kp)
                kp_scores.append(score['rouge-l']['f'])    

        topics.append(t)
        stances.append(s)
        scores.append(np.mean(kp_scores))
        
    
results_df = pd.DataFrame({"topic": topics, "stance": stances,  "r_scores": scores})
    

In [61]:
results_df.head(6)

,topic,stance,r_scores
0,Routine child vaccinations should be mandatory,-1,0.254200
1,Routine child vaccinations should be mandatory,1,0.314399
2,Social media platforms should be regulated by the government,-1,0.325476
3,Social media platforms should be regulated by the government,1,0.350095
4,The USA is a good country to live in,-1,0.184482
5,The USA is a good country to live in,1,0.223066


In [99]:
# key points in labeled ds are completely new and not another arguments like the way we do it, difficult to get good rouge score with our method.
test_ds_labeled[test_ds_labeled['argument'] == test_ds_labeled[test_ds_labeled['arg_id'] == 'arg_0_99']['key_point'].iloc[0]]

,idx,arg_id,argument,topic,stance,MACE-P,key_point_id,label,key_point,text,sentence1,sentence2,subset
